https://console.weaviate.cloud/

In [ ]:
!pip install weaviate-client
!pip install langchain
!pip install openai

In [ ]:
#OPENAI_API_KEY = ""
WEAVIATE_API_KEY = "API_KEY HERE"
WEAVIATE_CLUSTER = "https://kmpeq5s3ttezna2vwuwtwg.c0.asia-southeast1.gcp.weaviate.cloud"

## Data Reading

In [ ]:
!mkdir data

In [ ]:
!pip install unstructured
!pip install "unstructured[pdf]"
!pip install "unstructured[local]"

In [2]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader("./pdfs",glob = "**/*.pdf")
data = loader.load()

c:\Users\lshre\Gen-AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data

[Document(metadata={'source': 'pdfs\\OWASP_LLMGenAI_SecuritySolutions_ReferenceGuide.pdf'}, page_content='| GenAI(cid:428) LLMSecOps and Sec(cid:311)ri(cid:308)(cid:345) Sol(cid:311)(cid:308)ion Landscape\n\nP(cid:311)blished(cid:429) (cid:371)(cid:370)(cid:440)(cid:372)(cid:374)(cid:440)(cid:372)(cid:374)\n\nRe(cid:745)i(cid:742)ion Hi(cid:742)(cid:743)or(cid:748)\n\nRevision\n\nDate\n\nAuthors\n\nDescription\n\n(cid:427)01\n\n6(cid:440)4(cid:440)2024\n\nScott Clinton\n\nInitial Draft(cid:428) Charter\n\n(cid:427)05\n\n8(cid:440)10(cid:440)2024\n\nScott Clinton(cid:428) Contributors Inputs\n\nUpdated with initial feedback\n\n(cid:427)06\n\n10(cid:440)15(cid:440)2024\n\nScott Clinton(cid:428) Contributors(cid:428) Reviewer Inputs\n\nRe(cid:448)factor Solutions Landscape categories(cid:428)\n\n1(cid:427)0\n\n10(cid:440)15(cid:440)2024\n\nContributors(cid:428) Reviewers\n\nFinal Release Candidate\n\nThe information provided in this document does not(cid:428) and is not intended to(cid:42

## Text Splitting

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(data)

In [5]:
docs

[Document(metadata={'source': 'pdfs\\OWASP_LLMGenAI_SecuritySolutions_ReferenceGuide.pdf'}, page_content='| GenAI(cid:428) LLMSecOps and Sec(cid:311)ri(cid:308)(cid:345) Sol(cid:311)(cid:308)ion Landscape\n\nP(cid:311)blished(cid:429) (cid:371)(cid:370)(cid:440)(cid:372)(cid:374)(cid:440)(cid:372)(cid:374)\n\nRe(cid:745)i(cid:742)ion Hi(cid:742)(cid:743)or(cid:748)\n\nRevision\n\nDate\n\nAuthors\n\nDescription\n\n(cid:427)01\n\n6(cid:440)4(cid:440)2024\n\nScott Clinton\n\nInitial Draft(cid:428) Charter\n\n(cid:427)05\n\n8(cid:440)10(cid:440)2024\n\nScott Clinton(cid:428) Contributors Inputs\n\nUpdated with initial feedback\n\n(cid:427)06\n\n10(cid:440)15(cid:440)2024\n\nScott Clinton(cid:428) Contributors(cid:428) Reviewer Inputs\n\nRe(cid:448)factor Solutions Landscape categories(cid:428)\n\n1(cid:427)0\n\n10(cid:440)15(cid:440)2024\n\nContributors(cid:428) Reviewers\n\nFinal Release Candidate\n\nThe information provided in this document does not(cid:428) and is not intended to(cid:42

In [6]:
len(docs)

119

## Embedding Convertion

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key= OPENAI_API_KEY)

In [ ]:
embeddings

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

C:\Users\lshre\AppData\Local\Temp\ipykernel_6884\3778441718.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


In [8]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='intfloat/multilingual-e5-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Vector Database Storage

In [22]:
import weaviate
#fom langchain.vectorstores import Weaviate
from weaviate.classes.init import Auth

#Connect to weaviate Cluster
auth_config = weaviate.auth.AuthApiKey(api_key = WEAVIATE_API_KEY)
WEAVIATE_URL = WEAVIATE_CLUSTER

client = weaviate.WeaviateClient(
    url = WEAVIATE_URL,
   
    auth_client_secret = auth_config,
    startup_period = 10
)

TypeError: WeaviateClient.__init__() got an unexpected keyword argument 'url'

In [ ]:
client.is_ready()

In [ ]:
# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])

In [ ]:
# load text into the vectorstore
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)

## Similarity Measurement

In [ ]:
query = "what is a yolo?"

# retrieve text related to the query
docs = vectorstore.similarity_search(query, top_k=20)

In [ ]:
docs

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
# define chain
chain = load_qa_chain(
    OpenAI(openai_api_key = OPENAI_API_KEY,temperature=0),
    chain_type="stuff")

In [ ]:
# create answer
chain.run(input_documents=docs, question=query)